This is a notebook to generate all the prompts and seeds in the prompts google sheet

In [2]:
song_name = 'pipey' #@param {type:"string"}

import os
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')

code_folder = '/content/gdrive/MyDrive/AI-Art'
fp = os.path.join(code_folder, 'input_data.xlsx')

df_prompt = pd.read_excel(fp, 'prompts_{}'.format(song_name), index_col=0).dropna(how='all')
df_prompt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,prompt,seeds,guidance_scale
name,,,
sea robot siege 1,"army of various robot sea monsters, catapults ...",5182330954752209 334184349241288,7.5
sea robot siege 2,"robot armies fighting with laser pistols, sytl...",1878558593600852,7.5
sea robot siege 3,"robot armies fighting with laser pistols, sytl...",684805382478854,7.5
sea robot siege 4,"blue tidal waves in the distance, robot army, ...",7575365931752249,7.5
roiling sea,realistic photography art of the ocean zoomed ...,241427415206362 8525396860514958,7.5
red eye depth,realistic photography art of the ocean zoomed ...,7903125207733128,7.5
red eye depth2,realistic photography art of the ocean zoomed ...,3489535073209514 352836976540513,7.5
red eye depth3,realistic photography of the ocean zoomed in o...,4928293349930688,50.0
red eye depth4,realistic photography comic art of the ocean z...,8961019198724153 1613804757336012,25.0


In [3]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate

In [4]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")



Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Iterate through prompts and seeds, outputting an image for both

In [5]:
device = "cuda"
generator = torch.Generator(device=device)
width = 512
height = 512
skip_existing = True

output_folder = os.path.join(code_folder, 'prompt_images', song_name)

if not os.path.exists(output_folder): os.makedirs(output_folder)

for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(' ')
  prompt = row['prompt']
  guidance_scale = float(row['guidance_scale'])

  for seed in seeds:
    output_fn = "{}_{}.png".format(name, seed)

    if os.path.exists(os.path.join(output_folder, output_fn)):
      if skip_existing:
        print("{} already exists, skipping".format(output_fn))
        continue

    generator.manual_seed(int(seed))

    latent = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )

    with torch.autocast(device):
      images = pipe(
          prompt,
          guidance_scale=guidance_scale,
          latents = latent,
      )

    output_image = images.images[0]

    output_image.save(os.path.join(output_folder, output_fn))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 11>:33                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/utils/_contextlib.py:115 in decorate_context        │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diff │
│ usion.py:680 in __call__                                                                         │
│                                                                                                  │
│   677 │   │   │   │   │   noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text -   │
│   678 │   │   │   │                                                                              │
│   679 │   │   │   │   # compute the previous noisy sample x_t -> x_t-1                           │
│ ❱ 680 │   │   │   │   latents = self.scheduler.step(noise_pred, t, latents, **extra_step_kwarg   │
│   681 │   │   │   │                                                                              │
│   682 │   │   │   │   # call the callback, if provided                                           │
│   683 │   │   │   │   if i == len(timesteps) - 1 or ((i + 1) > num_warmup_steps and (i + 1) %    │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/diffusers/schedulers/scheduling_pndm.py:221 in step       │
│                                                                                                  │
│   218 │   │   if self.counter < len(self.prk_timesteps) and not self.config.skip_prk_steps:      │
│   219 │   │   │   return self.step_prk(model_output=model_output, timestep=timestep, sample=sa   │
│   220 │   │   else:                                                                              │
│ ❱ 221 │   │   │   return self.step_plms(model_output=model_output, timestep=timestep, sample=s   │
│   222 │                                                                                          │
│   223 │   def step_prk(                                                                          │
│   224 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/diffusers/schedulers/scheduling_pndm.py:337 in step_plms  │
│                                                                                                  │
│   334 │   │   else:                                                                              │
│   335 │   │   │   model_output = (1 / 24) * (55 * self.ets[-1] - 59 * self.ets[-2] + 37 * self   │
│   336 │   │                                                                                      │
│ ❱ 337 │   │   prev_sample = self._get_prev_sample(sample, t